# converting wind files into NetCDF

It's easiest if wind data is converted into a constant file type, so NetCDF it is.

Different sections of the notebook will have different code on converting files from different sources.

The simulations only support NetCDF data right now, sooooooo

## from txt file from national data buoy center

https://www.ndbc.noaa.gov/

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

infile = "data/field_netcdfs/ndbc_46025_wind.txt"
outfile = "data/field_netcdfs/ndbc_46025_wind.nc"

In [ ]:
df = pd.read_csv(infile, r" +")
# units row, drop
df = df.drop(0)
df = df.drop(df[df["WDIR"] == "MM"].index)
df

In [ ]:
new_df["time"].shape

In [ ]:
# notes: wind direction is where wind is coming from, pointing in opposite direction
# 3% of wspd added to ocean vector
uw_obs = -df["WSPD"].astype(float).values * np.sin(
    np.deg2rad(df["WDIR"].astype(float).values)
)
vw_obs = -df["WSPD"].astype(float).values * np.cos(
    np.deg2rad(df["WDIR"].astype(float).values)
)

# the column names are different oh god why
timestamp = (
    df["%YY"] + "-" + df["MM"] + "-" + df["DD"] + "T" + df["hh"] + ":" + df["mm"]
)
timestamp = pd.to_datetime(timestamp)

new_df = pd.DataFrame({"time": timestamp, "u": uw_obs, "v": vw_obs})
new_df = new_df.sort_values(by="time")

u = xr.DataArray(new_df["u"], coords={"time": new_df["time"]})
v = xr.DataArray(new_df["v"], coords={"time": new_df["time"]})

ds = xr.Dataset({"U": u, "V": v})
ds

In [ ]:
ds.to_netcdf(outfile)

## converting from .mat file lol

generated mat files from GFS data will have U and V components created from John's code

In [ ]:
from scipy.io import loadmat
import numpy as np
import xarray as xr
import datetime

infile = "data/field_netcdfs/GFS_PointSource_winds_188_3325.mat"
outfile = "data/field_netcdfs/GFS_PointSource_Winds_188_3325.nc"

In [ ]:
winds = loadmat(infile)
converted = [datetime.datetime.strptime(t, "%m-%d-%Y %H:%M") for t in winds["time_str"]]
times = np.array(converted, dtype=np.datetime64)
u = xr.DataArray(winds["u"].squeeze(), coords={"time": times})
v = xr.DataArray(winds["v"].squeeze(), coords={"time": times})
ds = xr.Dataset({"U": u, "V": v})
ds

In [ ]:
ds.to_netcdf(outfile)